In [189]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import requests as req
import plotly.graph_objects as go

# show all columns when printed
pd.set_option('display.max_columns', None)

In [190]:
df = pd.read_excel('./NikeFactoryLocations.xlsx')

In [191]:
# drop the unnecessary column
df = df.drop(['Operating Region'], axis=1)

In [197]:
def float_to_int(df, cols):
    df[cols] = df[cols].fillna(0.0).astype(int)

In [198]:
# change float columns to int (to original type)
cols = ['Total Workers', 'Line Workers']
# df[cols] = df[cols].fillna(0.0).astype(int)
# df.info()
float_to_int(df, cols)

In [201]:
# create Female Worker and Migrant Worker count columns
df['Female Worker Count'] = df['Total Workers'] * df['% Female Workers']
df['Migrant Worker Count'] = df['Total Workers'] * df['% Migrant Workers']
cols = ['Female Worker Count', 'Migrant Worker Count']
float_to_int(df, cols)

In [202]:
df.head(10)

,Factory Name,Factory Type,Product Type,"Nike, Inc. Brand(s)",Events,Supplier Group,Address,City,State,Postal Code,Country,Total Workers,Line Workers,% Female Workers,% Migrant Workers,Contact Name,Contact Phone,Contact Fax,Contact Email,Subcons,Female Worker Count,Migrant Worker Count
0,"A & K DESIGNS, INC.",FINISHED GOODS,APPAREL,NIKE,Collegiate,A & K DESIGNS,8325 SE HARNEY ST.,PORTLAND,OREGON,97266,USA,94,82,0.69,0.000,Paul Lee,503-238-6514,877-968-2889,paul.lee@akdesignsinc.com,NaN,64,0
1,A. FIRST VINA CO. LTD.,FINISHED GOODS,APPAREL,NIKE,NaN,A. FIRST VINA CO. LTD.,TAM PHUOC INDUSTRIAL ZONE,BIEN HOA,DONG NAI,NaN,VIETNAM,1958,1844,0.88,0.062,NaN,NaN,NaN,NaN,NaN,1723,121
2,"ACABADOS Y SERIGRAFIA, SA (ACSA)",FINISHED GOODS,APPAREL,NIKE,NaN,NO SUPPLIER GROUP,KM.16.5 A SAN JUAN SACATEPEQUEZ,MIXCO,GUATEMALA,NaN,GUATEMALA,191,181,0.32,0.000,NaN,NaN,NaN,NaN,NaN,61,0
3,ADORA FOOTWEAR LIMITED,FINISHED GOODS,FOOTWEAR,CONVERSE,NaN,HONG FU,TAM DIEP INDUSTRIAL ZONE,TAM DIEP TOWN,NINH BINH PROVINCE.,NaN,VIETNAM,7795,6899,0.91,0.005,NaN,NaN,NaN,NaN,NaN,7093,38
4,AJARA TEXTILE,FINISHED GOODS,APPAREL,NIKE,NaN,MILTEKS,31 MOTSERELIA STR.,POTI,SAMEGRELO-ZEMO SVANETI,4400,GEORGIA,616,582,0.96,0.000,NaN,NaN,NaN,NaN,NaN,591,0
5,ALL WELLS INTERNATIONAL,FINISHED GOODS,EQUIPMENT,NIKE,NaN,ALL WELLS INTERNATIONAL,TOC TIEN COMMUNE TAN THANH DISTRICT,BA RIA,VUNG TAU PROVINCE,NaN,VIETNAM,5313,3678,0.80,0.318,NaN,NaN,NaN,NaN,NaN,4250,1689
6,ANDBRAVA SRL,FINISHED GOODS,EQUIPMENT,CONVERSE,NaN,NO SUPPLIER GROUP,"SARGENTO CABRAL - PASAJE 62, Nº 1071",BUENOS AIRES,BUENOS AIRES,"Pasaje 62, Nº 1071",ARGENTINA,7,5,0.14,0.000,NaN,NaN,NaN,NaN,NaN,0,0
7,ANHUI COSMO SPORTING GOODS COMPANY LIMITED,FINISHED GOODS,EQUIPMENT,NIKE,NaN,ANHUI COSMO,"ECONOMIC DEVELOPMENT ZONE, JINZHAI COUNTY",LU'AN CITY,ANHUI,NaN,CHINA,372,308,0.89,0.000,NaN,NaN,NaN,NaN,NaN,331,0
8,ANIGER CALÇADOS SUPRIMENTOS E EMPREENDIMENTOS ...,FINISHED GOODS,FOOTWEAR,NIKE,NaN,ANIGER,"RUA ARMINDO ELTZ, 51 - QUATRO COLÔNIAS",CAMPO BOM – RS,RS,93700-000,BRAZIL,490,218,0.49,0.000,NaN,NaN,NaN,NaN,"A & K Designs, Inc., Acabados y Serigrafia, SA...",240,0
9,ANNIC LLC,FINISHED GOODS,APPAREL,NIKE,Collegiate,GILDAN,"KM45 1/2, CARRETERA MASATEPE-SAN MARCOS,",MASATEPE,NaN,NaN,NICARAGUA,3963,3398,0.50,0.002,Lesbia Elisa Chavarria,505-2532-6000,NaN,lchavarria@gildan.com,NaN,1981,7


In [164]:
API_KEY='26d87b67e9196fced6ce50d77eb89968'

In [165]:
df['Longitude'], df['Latitude'] = np.nan, np.nan

# get the geolocation info via OpenWeatherMap API
for i in range(len(df)):
    city = df['City'][i]
    res = req.get(f'https://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={API_KEY}')
    if len(res.json()) > 0:
        df['Longitude'].iloc[i] = res.json()[0]['lon']
        df['Latitude'][i] = res.json()[0]['lat']
    else:
        pass

/var/folders/0k/369f7zf50c59w8dkhr5_kczr0000gn/T/ipykernel_10001/659302689.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/0k/369f7zf50c59w8dkhr5_kczr0000gn/T/ipykernel_10001/659302689.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [166]:
# geomap of factories
fig = go.Figure(data=go.Scattergeo(
        lon = df['Longitude'],
        lat = df['Latitude'],
        text = df['City'],
        mode = 'markers'
        ))

fig.update_layout(
        title = 'Nike Factory Locations all around the World',
        geo_scope='world',
    )
fig.show()

In [167]:
# bubble chart
fig = px.scatter(df, x="Total Workers", y="State", color="Country", size="Total Workers",
                 hover_name="State", log_x=True, size_max=60)
fig.show()

In [168]:
# pie chart
fig = px.pie(df, values='Total Workers', names='Country', title='Total Worker Distrubution by Country')
fig.update_traces(textposition='inside', textinfo='percent')
fig.show()

In [178]:
# bar chart
fig = px.bar(df, x="Product Type", y="Country", color="Country", title="Product Type by Countries")
fig.show()

In [219]:
# female/migrant worker distrubitions over factories mentioned with the Brand together
fig = px.bar(df, x='Factory Type', y='Total Workers',
             hover_data=['Female Worker Count', 'Migrant Worker Count'], color="Nike, Inc. Brand(s)",
            #  labels={'':''}, 
             height=400)
fig.show()